In [1]:
!pip install -q scikit-learn numpy pandas altair kaggle==1.6.6 graphviz imbalanced-learn
!pip install -q xgboost lightgbm catboost statsmodels plotly optuna tensorflow

In [2]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#최대한 간결하게 적느라 빼놓았지만,
#.head(), .info(), .value_counts(), .isnull().sum() 등을 활용하여
#전처리 후 데이터가 어떻게 변화되는지 확인하는 것이 좋을 것 같습니다!

In [5]:
#한글 폰트(아래 feature importance 그래프에서 한글이 깨지는 것을 방지하기 위해)
plt.rc('font', family='NanumBarunGothic')

# Data Load

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
train = pd.read_csv('/content/drive/My Drive/Aimers6/train.csv').drop(columns=['ID'])
test = pd.read_csv('/content/drive/My Drive/Aimers6/test.csv').drop(columns=['ID'])

In [8]:
X = train #.drop('임신 성공 여부', axis=1) #몇몇 피처는 target 인코딩 할 것이기 때문에, 그 후에 드랍하였습니다.
y = train['임신 성공 여부']

# Data Pre-processing

In [9]:
# "알 수 없음"을 "만43-44세"로 변경하였는데, 다른 나이 구간으로 해보셔도 될 것 같습니다.
X['시술 당시 나이'] = X['시술 당시 나이'].replace('알 수 없음', '만43-44세')
test['시술 당시 나이'] = test['시술 당시 나이'].replace('알 수 없음', '만43-44세')

# Target Encoding 적용
age_success_rate = X.groupby("시술 당시 나이")["임신 성공 여부"].mean()
X['시술 당시 나이'] = X['시술 당시 나이'].map(age_success_rate)
test['시술 당시 나이'] = test['시술 당시 나이'].map(age_success_rate)


In [10]:
# 배아 생성 주요 이유 그룹화 (Treatment, Success, Fail)           #EDA 결과 임신 성공률에 차이가 나서 그룹화 해보았음.
reason_mapping = {
    '현재 시술용': 'Treatment',
    '기증용, 배아 저장용, 현재 시술용': 'Success',
    '기증용, 현재 시술용': 'Success',
    '난자 저장용, 현재 시술용': 'Success',
    '배아 저장용, 현재 시술용': 'Success',

    '기증용': 'Fail',
    '기증용, 난자 저장용': 'Fail',
    '기증용, 배아 저장용': 'Fail',
    '난자 저장용, 배아 저장용': 'Fail',
    '난자 저장용': 'Fail',
    '배아 저장용': 'Fail',
    '연구용, 배아 저장용, 연구용': 'Fail',
    '배아 저장용, 연구용, 현재 시술용': 'Fail'
}

# 데이터 변환 적용
X['배아 생성 주요 이유'] = X['배아 생성 주요 이유'].map(reason_mapping).fillna('Treatment')

# 변환 결과 확인
print(X['배아 생성 주요 이유'].value_counts())


배아 생성 주요 이유
Treatment    240025
Fail          12434
Success        3892
Name: count, dtype: int64


In [11]:
# 각 그룹별 평균 임신 성공률 계산
success_rate = X.groupby("배아 생성 주요 이유")["임신 성공 여부"].mean()

# Target Encoding 적용
X['배아 생성 주요 이유'] = X['배아 생성 주요 이유'].map(success_rate)

In [12]:
#test에도 똑같이 적용
test['배아 생성 주요 이유'] = test['배아 생성 주요 이유'].map(reason_mapping).fillna('Treatment')
test['배아 생성 주요 이유'] = test['배아 생성 주요 이유'].map(success_rate)


In [13]:
# '난자 출처' 피처의 '알 수 없음' 값을 '본인 제공' 값으로(최빈값으로) 변환
X['난자 출처'] = X['난자 출처'].replace('알 수 없음', '본인 제공')

# 결과 확인
print(X['난자 출처'].value_counts())

난자 출처
본인 제공    240582
기증 제공     15769
Name: count, dtype: int64


In [14]:
test['난자 출처'] = test['난자 출처'].replace('알 수 없음', '본인 제공')

print(test['난자 출처'].value_counts())

난자 출처
본인 제공    84573
기증 제공     5494
Name: count, dtype: int64


In [15]:
X.loc[X["난자 출처"] == "본인 제공", "난자 출처"] = 0
X.loc[X["난자 출처"] == "기증 제공", "난자 출처"] = 1

X.loc[X["정자 출처"] == "배우자 제공", "정자 출처"] = 0
X.loc[X["정자 출처"] == "기증 제공", "정자 출처"] = 1
X.loc[X["정자 출처"] == "미할당", "정자 출처"] = 0
X.loc[X["정자 출처"] == "배우자 및 기증 제공", "정자 출처"] = 0

In [16]:
test.loc[test["난자 출처"] == "본인 제공", "난자 출처"] = 0
test.loc[test["난자 출처"] == "기증 제공", "난자 출처"] = 1

test.loc[test["정자 출처"] == "배우자 제공", "정자 출처"] = 0
test.loc[test["정자 출처"] == "기증 제공", "정자 출처"] = 1
test.loc[test["정자 출처"] == "미할당", "정자 출처"] = 0
test.loc[test["정자 출처"] == "배우자 및 기증 제공", "정자 출처"] = 0

In [17]:
X = X.drop([
    # '시술 시기 코드',
    '시술 유형',
    '특정 시술 유형',
    '배란 유도 유형',
    '단일 배아 이식 여부',
    '착상 전 유전 진단 사용 여부',
    '배아 생성 주요 이유',
    '난자 기증자 나이',
    '정자 기증자 나이',
    '클리닉 내 총 시술 횟수',
    'DI 시술 횟수',
    '총 임신 횟수',
    'DI 임신 횟수',
    '총 출산 횟수',
    'DI 출산 횟수',
    '총 생성 배아 수',
    '미세주입된 난자 수',
    '미세주입에서 생성된 배아 수',
    # '이식된 배아 수',
    '미세주입 배아 이식 수',
    '저장된 배아 수',
    '미세주입 후 저장된 배아 수',
    '해동된 배아 수',
    '저장된 신선 난자 수',
    '혼합된 난자 수',
    '기증자 정자와 혼합된 난자 수',
    '난자 기증자 나이',
    '정자 기증자 나이',
    '기증 배아 사용 여부',
    '미세주입된 난자 수',
    '임신 시도 또는 마지막 임신 경과 연수',
    '착상 전 유전 검사 사용 여부',
    'PGD 시술 여부',
    'PGS 시술 여부',
    '난자 채취 경과일',
    '난자 해동 경과일',
    '난자 혼합 경과일',
    '배아 이식 경과일',
    '배아 해동 경과일',
    '대리모 여부'
    ], axis=1)

In [18]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256351 entries, 0 to 256350
Data columns (total 33 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   시술 시기 코드            256351 non-null  object 
 1   시술 당시 나이            256351 non-null  float64
 2   배란 자극 여부            256351 non-null  int64  
 3   남성 주 불임 원인          256351 non-null  int64  
 4   남성 부 불임 원인          256351 non-null  int64  
 5   여성 주 불임 원인          256351 non-null  int64  
 6   여성 부 불임 원인          256351 non-null  int64  
 7   부부 주 불임 원인          256351 non-null  int64  
 8   부부 부 불임 원인          256351 non-null  int64  
 9   불명확 불임 원인           256351 non-null  int64  
 10  불임 원인 - 난관 질환       256351 non-null  int64  
 11  불임 원인 - 남성 요인       256351 non-null  int64  
 12  불임 원인 - 배란 장애       256351 non-null  int64  
 13  불임 원인 - 여성 요인       256351 non-null  int64  
 14  불임 원인 - 자궁경부 문제     256351 non-null  int64  
 15  불임 원인 - 자궁내막증       256351 non-nul

In [19]:
test = test.drop([
    # '시술 시기 코드',
    '시술 유형',
    '특정 시술 유형',
    '배란 유도 유형',
    '단일 배아 이식 여부',
    '착상 전 유전 진단 사용 여부',
    '배아 생성 주요 이유',
    '난자 기증자 나이',
    '정자 기증자 나이',
    '클리닉 내 총 시술 횟수',
    'DI 시술 횟수',
    '총 임신 횟수',
    'DI 임신 횟수',
    '총 출산 횟수',
    'DI 출산 횟수',
    '총 생성 배아 수',
    '미세주입된 난자 수',
    '미세주입에서 생성된 배아 수',
    # '이식된 배아 수',
    '미세주입 배아 이식 수',
    '저장된 배아 수',
    '미세주입 후 저장된 배아 수',
    '해동된 배아 수',
    '저장된 신선 난자 수',
    '혼합된 난자 수',
    '기증자 정자와 혼합된 난자 수',
    '난자 기증자 나이',
    '정자 기증자 나이',
    '기증 배아 사용 여부',
    '미세주입된 난자 수',
    '임신 시도 또는 마지막 임신 경과 연수',
    '착상 전 유전 검사 사용 여부',
    'PGD 시술 여부',
    'PGS 시술 여부',
    '난자 채취 경과일',
    '난자 해동 경과일',
    '난자 혼합 경과일',
    '배아 이식 경과일',
    '배아 해동 경과일',
    '대리모 여부'
    ], axis=1)

In [20]:
'''
# test = test.drop('불임 원인 - 여성 요인', axis=1)
test = test.drop('난자 채취 경과일', axis=1)
test = test.drop('난자 해동 경과일', axis=1)
test = test.drop('난자 혼합 경과일', axis=1)
test = test.drop('불임 원인 - 정자 면역학적 요인', axis=1)
test = test.drop('불임 원인 - 정자 운동성', axis=1)
test = test.drop('불임 원인 - 자궁경부 문제', axis=1)
test = test.drop('불임 원인 - 정자 형태', axis=1)
test = test.drop('배아 해동 경과일', axis=1)
test = test.drop('불임 원인 - 정자 농도', axis=1)
test = test.drop('시술 유형', axis=1)
test = test.drop('임신 시도 또는 마지막 임신 경과 연수', axis=1)
test = test.drop('착상 전 유전 검사 사용 여부', axis=1)
test = test.drop('PGD 시술 여부', axis=1)
test = test.drop('PGS 시술 여부', axis=1)
test = test.drop('배란 유도 유형', axis=1)
'''

"\n# test = test.drop('불임 원인 - 여성 요인', axis=1)\ntest = test.drop('난자 채취 경과일', axis=1)\ntest = test.drop('난자 해동 경과일', axis=1)\ntest = test.drop('난자 혼합 경과일', axis=1)\ntest = test.drop('불임 원인 - 정자 면역학적 요인', axis=1)\ntest = test.drop('불임 원인 - 정자 운동성', axis=1)\ntest = test.drop('불임 원인 - 자궁경부 문제', axis=1)\ntest = test.drop('불임 원인 - 정자 형태', axis=1)\ntest = test.drop('배아 해동 경과일', axis=1)\ntest = test.drop('불임 원인 - 정자 농도', axis=1)\ntest = test.drop('시술 유형', axis=1)\ntest = test.drop('임신 시도 또는 마지막 임신 경과 연수', axis=1)\ntest = test.drop('착상 전 유전 검사 사용 여부', axis=1)\ntest = test.drop('PGD 시술 여부', axis=1)\ntest = test.drop('PGS 시술 여부', axis=1)\ntest = test.drop('배란 유도 유형', axis=1)\n"

In [21]:
#'시술 시기 코드'에 One-Hot Encoding 적용
from sklearn.preprocessing import OneHotEncoder

# One-Hot Encoding 적용 열
one_hot_cols = ['시술 시기 코드']

# One-Hot Encoding 적용
one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Train 데이터 변환
onehot_encoded = one_hot_encoder.fit_transform(X[one_hot_cols])
onehot_encoded_df = pd.DataFrame(onehot_encoded, columns=one_hot_encoder.get_feature_names_out(one_hot_cols), index=X.index)

# 기존 데이터와 병합
X = pd.concat([X.drop(columns=one_hot_cols), onehot_encoded_df], axis=1)

In [22]:
#one-hot encoding
test_encoded = one_hot_encoder.transform(test[one_hot_cols])
test_encoded_df = pd.DataFrame(test_encoded, columns=one_hot_encoder.get_feature_names_out(one_hot_cols), index=test.index)

# 기존 데이터와 병합
test = pd.concat([test.drop(columns=one_hot_cols), test_encoded_df], axis=1)

In [23]:
categorical_columns = [
    # "시술 시기 코드", #one-hot
    "시술 당시 나이", #target-encoder
    # "시술 유형", #삭제
    # "특정 시술 유형", #삭제
    # "배란 자극 여부", #삭제
    # "배란 유도 유형", #삭제
    # "단일 배아 이식 여부", #삭제
    # "착상 전 유전 검사 사용 여부", #삭제
    # "착상 전 유전 진단 사용 여부", #ordinal
    "남성 주 불임 원인", #ordinal
    "남성 부 불임 원인", #ordinal
    "여성 주 불임 원인", #ordinal
    "여성 부 불임 원인", #ordinal
    "부부 주 불임 원인", #ordinal
    "부부 부 불임 원인", #ordinal
    "불명확 불임 원인", #ordinal
    "불임 원인 - 난관 질환", #ordinal
    "불임 원인 - 남성 요인", #ordinal
    "불임 원인 - 배란 장애", #ordinal
    # "불임 원인 - 여성 요인", #삭제
    # "불임 원인 - 자궁경부 문제", #삭제
    "불임 원인 - 자궁내막증", #ordinal
    # "불임 원인 - 정자 농도", #삭제
    # "불임 원인 - 정자 면역학적 요인", #삭제
    # "불임 원인 - 정자 운동성", #삭제
    # "불임 원인 - 정자 형태", #삭제
    # "배아 생성 주요 이유", #그룹화, target-encoder
    "총 시술 횟수", #ordinal, 추후에 target 고민
    # "클리닉 내 총 시술 횟수", #삭제
    "IVF 시술 횟수", #ordinal, 추후에 target 고민
    # "DI 시술 횟수", #ordinal
    # "총 임신 횟수", #ordinal
    "IVF 임신 횟수", #ordinal
    # "DI 임신 횟수", #ordinal
    # "총 출산 횟수", #ordinal
    "IVF 출산 횟수", #ordinal
    # "DI 출산 횟수", #ordinal
    # "난자 출처", #ordinal, '알 수 없음' -> '본인 제공'
    # "정자 출처", #ordinal
    # "난자 기증자 나이", #ordinal
    # "정자 기증자 나이", #ordinal
    "동결 배아 사용 여부", #ordinal, 삭제 고민
    "신선 배아 사용 여부",#ordinal, 삭제 고민
    # "기증 배아 사용 여부",#ordinal, 삭제 고민
    # "대리모 여부",#ordinal, 삭제 고민
    # "PGD 시술 여부", #삭제
    # "PGS 시술 여부" #삭제
]

In [24]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [25]:
#Ordinal Encoding 적용
from sklearn.preprocessing import  OrdinalEncoder

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [26]:
#수치형 칼럼

numeric_columns = [
    # "임신 시도 또는 마지막 임신 경과 연수", #삭제
    # "총 생성 배아 수", #결측치 0 대체
    # "미세주입된 난자 수", #결측치 0 대체
    # "미세주입에서 생성된 배아 수", #결측치 0 대체
    # "이식된 배아 수", #결측치 2로 대체
    # "미세주입 배아 이식 수", #결측치 0 대체
    # "저장된 배아 수", #결측치 0 대체
    # "미세주입 후 저장된 배아 수", #결측치 0 대체
    # "해동된 배아 수", #결측치 0 대체
    "해동 난자 수", #결측치 0 대체
    "수집된 신선 난자 수", #결측치 0 대체
    # "저장된 신선 난자 수", #결측치 0 대체
    # "혼합된 난자 수", #결측치 0 대체
    "파트너 정자와 혼합된 난자 수", #결측치 0 대체
    # "기증자 정자와 혼합된 난자 수", #결측치 0 대체
    # "난자 채취 경과일", #삭제
    # "난자 해동 경과일", #삭제
    # "난자 혼합 경과일", #삭제
    # "배아 이식 경과일",
    # "배아 해동 경과일" #삭제
]

In [27]:
#"이식된 배아 수"는 최빈값이 2이므로 따로 빼서 결측치를 2로 대체
numeric_fillna_2 = [
    "이식된 배아 수"
]

In [28]:
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0) #나머지
X_train_encoded[numeric_fillna_2] = X_train_encoded[numeric_fillna_2].fillna(2) #이식된 배아 수

X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_fillna_2] = X_test_encoded[numeric_fillna_2].fillna(2)

In [29]:
X_train_encoded = X_train_encoded.drop(columns=['임신 성공 여부']) #처음에 안빼줬던 target 변수 빼주기

# Model Training

## RELU

In [30]:
X_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256351 entries, 0 to 256350
Data columns (total 38 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   시술 당시 나이            256351 non-null  float64
 1   배란 자극 여부            256351 non-null  int64  
 2   남성 주 불임 원인          256351 non-null  float64
 3   남성 부 불임 원인          256351 non-null  float64
 4   여성 주 불임 원인          256351 non-null  float64
 5   여성 부 불임 원인          256351 non-null  float64
 6   부부 주 불임 원인          256351 non-null  float64
 7   부부 부 불임 원인          256351 non-null  float64
 8   불명확 불임 원인           256351 non-null  float64
 9   불임 원인 - 난관 질환       256351 non-null  float64
 10  불임 원인 - 남성 요인       256351 non-null  float64
 11  불임 원인 - 배란 장애       256351 non-null  float64
 12  불임 원인 - 여성 요인       256351 non-null  int64  
 13  불임 원인 - 자궁경부 문제     256351 non-null  int64  
 14  불임 원인 - 자궁내막증       256351 non-null  float64
 15  불임 원인 - 정자 농도       256351 non-nul

In [33]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Model Architecture
model = Sequential([
    Dense(38, activation='relu', input_shape=(38,)),  # Input Layer
    # Dense(32, activation='relu'),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dropout(0.1),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Output Layer
])

# Training Parameters
input_size = (None, 38)  # (batch_size, input_features) - batch_size is inferred during training
output_size = (None, 1)  # (batch_size, output_units)
total_dense_layers = 9
regularization = "Batch Normalization, Dropout"
batch_size = 128
optimizer = Adam()
loss = 'binary_crossentropy'
epochs = 50
callbacks = [EarlyStopping(patience=5)]  # Early Stopping callback

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Print model summary
model.summary()

# Dummy data for demonstration (replace with your actual data)
import numpy as np
X_train = np.random.rand(1000, 38)  # 1000 samples, 25 features
y_train = np.random.randint(0, 2, size=(1000, 1))  # Binary classification labels

# Train the model
model.fit(X_train_encoded, y, epochs=epochs, batch_size=batch_size, callbacks=callbacks, validation_split=0.2)

# Evaluate the model (optional)
# loss, accuracy = model.evaluate(X_test, y_test)
# print(f"Loss: {loss}, Accuracy: {accuracy}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 38)                  │           1,482 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           2,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 352,523 (1.34 MB)

 Trainable params: 352,011 (1.34 MB)

 Non-trainable params: 512 (2.00 KB)

Epoch 1/50
1603/1603 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - accuracy: 0.7399 - loss: 0.5288 - val_accuracy: 0.7417 - val_loss: 0.5023
Epoch 2/50
1603/1603 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7429 - loss: 0.5029 - val_accuracy: 0.7417 - val_loss: 0.5052
Epoch 3/50
1603/1603 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.7422 - loss: 0.5018 - val_accuracy: 0.7418 - val_loss: 0.4988
Epoch 4/50
1603/1603 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.7407 - loss: 0.5030 - val_accuracy: 0.7416 - val_loss: 0.4997
Epoch 5/50
1603/1603 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7407 - loss: 0.5021 - val_accuracy: 0.7418 - val_loss: 0.4984
Epoch 6/50
1603/1603 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7421 - loss: 0.5012 - val_accuracy: 0.7418 - val_loss: 0.4990
Epoch 7/50
1603/1603 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.7410 - loss: 0.5016 - val_accuracy: 0.7418 - val_loss: 0.4982
Epoch 8/50
1603/1603 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7421 - loss: 0.499

In [36]:
'''
from sklearn.model_selection import StratifiedKFold, cross_val_score

# Stratified K-Fold 설정 (데이터 불균형 고려)
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

roc_auc_scores = cross_val_score(model, X_train_encoded, y, cv=skf, scoring='roc_auc')

# 결과 출력
print(f"ROC-AUC scores: {roc_auc_scores}")
print(f"Mean ROC-AUC score: {roc_auc_scores.mean()}")
'''

'\nfrom sklearn.model_selection import StratifiedKFold, cross_val_score\n\n# Stratified K-Fold 설정 (데이터 불균형 고려)\nskf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)\n\nroc_auc_scores = cross_val_score(model, X_train_encoded, y, cv=skf, scoring=\'roc_auc\')\n\n# 결과 출력\nprint(f"ROC-AUC scores: {roc_auc_scores}")\nprint(f"Mean ROC-AUC score: {roc_auc_scores.mean()}")\n'

# Predict

In [35]:
y_test = model.predict(X_test_encoded) #다른 모델 예측하려면 cat_model을 다른 모델명으로 바꿔주면 됨.

2815/2815 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step


# Submission


In [37]:
sample_submission = pd.read_csv('/content/drive/My Drive/Aimers6/sample_submission.csv')
sample_submission['probability'] = y_test

sample_submission.to_csv('/content/drive/My Drive/Aimers6/relu_submission.csv', index=False)